In [1]:
import sys; sys.path.insert(0, '../ia342-dynamic-programming/') # add parent folder path where lib folder

import sys; sys.path.insert(0, '../ia342-dynamic-programming/') # add parent folder path where lib folder

# IA342 Tópicos em Otimização de Sistemas III
## Programação Dinâmica, Variações e Aproximações


### Lista de Exercícios e Trabalhos

#### Lista 1


Para a solução da Lista 1 foi criada uma classe abstrata, [DiscreteDynamicProblem](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/discreteDynamicProblem.py), com alguns métodos que precisam ser implementados conforme o problema que deseja-se resolver, um método de solve e um que calcula a trajetória ótima.

O método de [solve](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/discreteDynamicProblem.py#L137) foi implementado como pode ser visto abaixo:

---
```
def solve(self):
        """
        Solves the optimality recursive equation using a backward strategy
        """
        for xk in self.state(self.numberOfStages()):
            self.__F[self.numberOfStages(), xk] = self.finalStateCost(xk)
        
        for k in self.__stagesGenerator(): #n-1 to 0
            for xk in self.state(k):
                F_aux = self.inf()
                u_aux = None
                # Calculates the best decision on stage k and state uk
                for uk in self.decision(k):
                    xk_next = self.transitionFunction(k, xk, uk)
                    F_aux_uk = self.elementaryCost(k, xk, uk) + self.F(k+1, xk_next)

                    if F_aux > F_aux_uk:
                        F_aux = F_aux_uk
                        u_aux = uk
                self.__F[k, xk] = F_aux
                self.__policy[k, xk] = u_aux
```
---

Além disso foi implementado um [método](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/discreteDynamicProblem.py#L159) que calcula a trajetória ótima, dado um stado inicial:

---

```
def optimalTrajectory(self, initialState:np.array):
        """
        Calculates the optimal trajectory given a initial state

        Args:
            initialState (np.array): initial state to calculate the optimal trajectory

        Returns:
            u_optimal: states of the optimal trajectory
            policy_optimal: decisions for the optimal trajectory
        """
        u_optimal = [initialState]
        policy_optimal = list()
        for k in range(self.numberOfStages()):
            policy_optimal.append(self.policy(k, u_optimal[-1]))
            u_optimal.append(self.transitionFunction(k, u_optimal[-1], policy_optimal[-1]))
        
        return (u_optimal, policy_optimal, )
```

In [2]:
from dypro.dypro.discreteDynamicProblem import DiscreteDynamicProblem
from dypro.dypro.discreteDynamicProblem import generatorFromLIst
import numpy as np

##### Exercício 1 - Problema da fábrica de turbinas

Para a solução deste exercício foi necessário implementar os métodos abaixo, utilizando o método de solve da classe que ela implementa, conforme pode ser visto abaixo:

In [3]:
class SimpleProductionProblem(DiscreteDynamicProblem):
    def __init__(self, numberOfStages:int, initialState:np.array, demand, feasibleDecisions, feasibleStates, 
                finalStateCost, productionCost, currentStateCost, inf=np.inf, F=dict(), policy=dict()):
        self.__initialState = initialState
        self.__demand = demand
        self.__feasibleDecisions = feasibleDecisions
        self.__feasibleStates = feasibleStates
        self.__finalStateCost = finalStateCost
        self.__productionCost = productionCost
        self.__currentStateCost = currentStateCost
        super().__init__(numberOfStages, inf=inf, F=F, policy=policy)
        
    def state(self, k:int) -> np.array:
        return self.__feasibleStates(k)

    def decision(self, k:int) -> np.array:
        return self.__feasibleDecisions(k)

    def elementaryCost(self, k:int, state:np.array, decision:np.array) -> float:
        return self.__productionCost(k, decision) + self.__currentStateCost(k, state)

    def finalStateCost(self, state:np.array) -> float:
        return self.__finalStateCost(state)

    def transitionFunction(self, k:int, state:np.array, decision:np.array) -> np.array:
        return state + decision - self.__demand(k)

    def initialState(self) -> np.array:
        return self.__initialState
    

if __name__ == '__main__':
    
    STAGES = 4
    demandStage = lambda k: [2,1,1,0][k]
    stockCost = lambda k, state: [0, 3, 7][state] if state<3 else np.inf
    productionCost = lambda k, decision: [10, 17, 20][decision]
    stateSpace = lambda k: generatorFromLIst([0, 1, 2])
    productionSpace = lambda k: generatorFromLIst([0, 1, 2])
    finalState = lambda state: 0 if state==1 else np.inf
    initialStock = 1
    
    dp = SimpleProductionProblem(STAGES, initialStock, demandStage, productionSpace, stateSpace, 
                                 finalState, productionCost, stockCost)
    
    dp.solve()
    u_optimal, policy_optimal = dp.optimalTrajectory(initialStock)

    print(f"states of the optimal trajectory: {u_optimal}")
    print(f"Decisions for the optimal trajectory: {policy_optimal}")
    print(f"Cost of the optimal trajectory: {dp.F(0, initialStock)}")

states of the optimal trajectory: [1, 1, 0, 1, 1]
Decisions for the optimal trajectory: [2, 0, 2, 0]
Cost of the optimal trajectory: 69


##### Exercício 2 - Planejamento de produção com retardos

In [4]:
from dypro.dypro.discreteDynamicProblem import DiscreteDynamicProblem
from dypro.dypro.discreteDynamicProblem import generatorFromLIst, generatorFromLists
import numpy as np

class DelayProducerPlanning(DiscreteDynamicProblem):
    def __init__(self, numberOfStages:int, initialState:np.array, demand, feasibleDecisions, feasibleStates, 
                finalStateCost, productionCost, currentStateCost, changeStateCost, inf=np.inf, 
                 F=dict(), policy=dict()):
        self.__initialState = initialState
        self.__demand = demand
        self.__feasibleDecisions = feasibleDecisions
        self.__feasibleStates = feasibleStates
        self.__finalStateCost = finalStateCost
        self.__productionCost = productionCost
        self.__currentStateCost = currentStateCost
        self.__changeStateCost = changeStateCost
        super().__init__(numberOfStages, inf=inf, F=F, policy=policy)
        
    def state(self, k:int) -> np.array:
        return self.__feasibleStates(k)

    def decision(self, k:int) -> np.array:
        return self.__feasibleDecisions(k)

    def elementaryCost(self, k:int, state:np.array, decision:np.array) -> float:
        return self.__productionCost(k, decision) + self.__currentStateCost(k, state) + \
                self.__changeStateCost(k, state, decision)

    def finalStateCost(self, state:np.array) -> float:
        return self.__finalStateCost(state)

    def transitionFunction(self, k:int, state:np.array, decision:np.array) -> np.array:
        return (state[0] + decision - self.__demand(k), decision)

    def initialState(self) -> np.array:
        return self.__initialState
    

if __name__ == '__main__':
    
    STAGES = 3
    demandStage = lambda k: [2,4,1][k]
    stockCost = lambda k, state: state[0] if state[0]<=4 else np.inf
    changeProductionCost = lambda k, state, decision: max(decision - state[1], state[1] - decision)*2
    productionCost = lambda k, decision: ([3, 5, 3][k])*decision
    stateSpace = lambda k: generatorFromLists([0, 1, 2, 3, 4], [0, 1, 2, 3])
    productionSpace = lambda k: generatorFromLIst([0, 1, 2, 3])

    finalState = lambda state: 0
    initialState = (1, 1)
    
    dp = DelayProducerPlanning(STAGES, initialState, demandStage, productionSpace, stateSpace, 
                               finalState, productionCost, stockCost, changeProductionCost)
    dp.solve()
    u_optimal, policy_optimal = dp.optimalTrajectory(initialState)

    print(f"states of the optimal trajectory: {u_optimal}")
    print(f"Decisions for the optimal trajectory: {policy_optimal}")
    print(f"Cost of the optimal trajectory: {dp.F(0, initialState)}")

states of the optimal trajectory: [(1, 1), (2, 3), (0, 2), (0, 1)]
Decisions for the optimal trajectory: [3, 2, 1]
Cost of the optimal trajectory: 33


##### Exercício 4 - Problema de Planejamento de Produção com Perturbações Aleatórias

Para a solução de problemas com perturbações aleatórias da lista 1 foi criada a classe [StochasticDiscreteDynamicProblem](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/stochasticDiscreteDynamicProblem.py) com um método [solve](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/stochasticDiscreteDynamicProblem.py#L155) e mais alguns métodos que precisam ser implementados conforme o problema.

---

Segue o método solve:
```
def solve(self):
    """
    Solves the optimality recursive equation using a backward strategy
    """
    for xk in self.state(self.numberOfStages()):
        self.__F[self.numberOfStages(), xk] = self.finalStateCost(xk)

    for k in self.__stagesGenerator(): #n-1 to 0
        for xk in self.state(k):
            F_aux = self.inf()
            u_aux = None
            # Calculates the best decision on stage k and state uk
            for uk in self.decision(k):
                # Expected value of accumulated cost for decision uk and state xk
                E_F_aux = 0.0
                for wk in self.realizableRandomValues(k).randomValueIterator():
                    xk_next = self.transitionFunction(k, xk, uk, wk.getValue())
                    E_F_aux += wk.getProbability() * (self.elementaryCost(k, xk, uk, wk.getValue()) + \
                                self.F(k+1, xk_next))

                if F_aux > E_F_aux:
                    F_aux = E_F_aux
                    u_aux = uk
            self.__F[k, xk] = F_aux
            self.__policy[k, xk] = u_aux
```
---

Abaixo segue a implementação do problema utilizando essa classe:

In [5]:
from dypro.dypro.stochasticDiscreteDynamicProblem import StochasticDiscreteDynamicProblem, generatorFromLIst
from dypro.dypro.randomVariable import RandomVariable
import numpy as np

class SimpleStochasticDiscreteDynamicProblem(StochasticDiscreteDynamicProblem):
    def __init__(self, numberOfStages:int, feasibleStates, feasibleDecisions, productionCost, stockCost, initialState, 
                realizableRandomValues, finalStateCost, inf=100, F=dict(), policy=dict()):
        self.__feasibleStates = feasibleStates
        self.__feasibleDecisions = feasibleDecisions
        self.__productionCost = productionCost
        self.__stockCost = stockCost
        self.__initialState = initialState
        self.__realizableRandomValues = realizableRandomValues
        self.__finalStateCost = finalStateCost
        super().__init__(numberOfStages, inf, F, policy)

    def state(self, k:int) -> np.array:
        return self.__feasibleStates(k)

    def decision(self, k:int) -> np.array:
        return self.__feasibleDecisions(k)

    def elementaryCost(self, k:int, state:np.array, decision:np.array, random_variable:np.array) -> float:
        return self.__productionCost(k, decision) + self.__stockCost(k, state + decision - random_variable)

    def finalStateCost(self, state:np.array) -> float:
        return self.__finalStateCost(state)

    def __limit(self, v, min, max):
        if v<min:
            return min
        if v>max:
            return max
        return v

    def transitionFunction(self, k:int, state:np.array, decision:np.array, random_variable:np.array) -> np.array:
        return self.__limit(state + decision - random_variable, 0, 4)

    def initialState(self) -> np.array:
        return self.__initialState

    def realizableRandomValues(self, k:int) -> RandomVariable:
        return self.__realizableRandomValues(k)

if __name__ == '__main__':
    STAGES = 3

    # this cost works with the case that we cant meet the demand
    def stockCost(k, state):
        if state<0:
            return (-5*state)
        if state>4:
            return (state-4)*2 + 4
        else:
            return state
    
    productionCost = lambda k, decision: ([3, 5, 3][k])*decision
    
    stateSpace = lambda k: generatorFromLIst([0, 1, 2, 3, 4])
    productionSpace = lambda k: generatorFromLIst([0, 1, 2, 3])
    demandRandomValues = lambda k: ([
                                    RandomVariable((1,2,3), (0.2, 0.7, 0.1)),
                                    RandomVariable((3, 4, 5), (0.3, 0.6, 0.1)),
                                    RandomVariable((1, 2, 3), (0.5, 0.4, 0.1))
                                        ][k])

    finalStateCost = lambda state: 0
    initialState = 1

    a = SimpleStochasticDiscreteDynamicProblem(STAGES, stateSpace, productionSpace, productionCost, stockCost, initialState, demandRandomValues, finalStateCost)
    a.solve()

    print(f"All policy: {a.allPolicy()}")
    print(f"Expectancy of cost: {a.F(0, initialState)}")
    print(f"{a.accOptimalCost()}")

All policy: {(2, 0): 1, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (1, 0): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (0, 0): 3, (0, 1): 3, (0, 2): 3, (0, 3): 2, (0, 4): 1}
Expectancy of cost: 25.6
{(3, 0): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (3, 4): 0, (2, 0): 6.0, (2, 1): 3.0, (2, 2): 1.0, (2, 3): 1.4, (2, 4): 2.4, (1, 0): 25.0, (1, 1): 20.0, (1, 2): 14.999999999999998, (1, 3): 9.999999999999998, (1, 4): 5.9, (0, 0): 29.6, (0, 1): 25.6, (0, 2): 21.78, (0, 3): 18.78, (0, 4): 15.779999999999998}


### Trabalhos

#### Planejamentos de Produção de Energia Elétrica 1

Para resolver os problemas com uma única usina foram criadas três classes que implementam uma interface:

* [DiscreteDynamicProgramming](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/discreteDynamicProgramming.py#L5): implementa problemas de programação dinâmica
* [HydroelectricProduction](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/hydroelectricProduction.py#L3): implementa métodos associados à hidroelétricas
* [StochasticProblem](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/stochasticProblem.py#L5): implementa um método que devolve variáveis aleatórias

Além dessas classes foram criadas um método de solve para o [problema determinístico](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/solve.py#L8) e outro para o [problema com perturbação aleatória](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/solve.py#L104).

---

```
def solve(state, numberOfStages:int, finalStateCost, decision, solveInfeasibility, 
			transitionFunction, elementaryCost, initialFMap=dict(), initialPolicy=dict(), inf=np.inf):
	
	INFEASIBLE_MAP = dict()

	FMap = dict.copy(initialFMap)
	policy = dict.copy(initialPolicy)

	for xk in state(numberOfStages):
		FMap[numberOfStages, xk] = finalStateCost(xk)
		logging.debug(f"{FMap}")
	
	for k in range(numberOfStages-1, -1, -1): #n-1 to 0
		start_time = time.time()
		for xk in progressbar(state(k), prefix="states  >>>>>>" ,file=sys.stderr):
			F_aux = inf
			u_aux = None
			
			# Calculates the best decision on stage k and state uk
			for uk in decision(k):
				xk_next_maybe_infeasible = transitionFunction(k, xk, uk)
				

				if (k+1, xk_next_maybe_infeasible) not in FMap and \
                        (k+1, xk_next_maybe_infeasible) not in INFEASIBLE_MAP:
					xk_next, _F = solveInfeasibility(k+1, xk_next_maybe_infeasible, FMap)
					INFEASIBLE_MAP[k+1, xk_next_maybe_infeasible] = (xk_next, _F)

				elif (k+1, xk_next_maybe_infeasible) in INFEASIBLE_MAP:
					xk_next, _F = INFEASIBLE_MAP[k+1, xk_next_maybe_infeasible]

				else:
					xk_next, _F = (xk_next_maybe_infeasible, FMap[k+1, xk_next_maybe_infeasible])

				# F_aux_uk = elementaryCost(k, xk, uk) + FMap[k+1, xk_next]
				F_aux_uk = elementaryCost(k, xk, uk) + _F

				if F_aux > F_aux_uk:
					F_aux = F_aux_uk
					u_aux = uk
			
			FMap[k, xk] = F_aux
			policy[k, xk] = u_aux
		print(f"Stage {k} elapsed {time.time() - start_time} sec")
	
	return (FMap, policy)
```

---

##### Usina Hidroelétrica e Complementação Térmica
###### Usina de Ilha Solteira

A implementação da resolução deste problema encontra-se na seguinte classe: [HydroelectricProductionProblem](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/hydroelectricProductionProblem.py#L13).

In [6]:
from dypro.dypro.discreteDynamicProgramming import DiscreteDynamicProgramming
from dypro.dypro.hydroelectricProduction import HydroelectricProduction

from dypro.dypro.utils import limit, sampling, secondsOfMonth, nearestSample
from dypro.dypro.solve import solve, optimalTrajectory

import numpy as np

from time import time

import concurrent.futures

class HydroelectricProductionProblem(DiscreteDynamicProgramming, HydroelectricProduction):
    def __init__(self, numberOfStages:int, year:int, maxFlow, stateSampling=100, decisionSampling=100, inf=np.inf):
        DiscreteDynamicProgramming.__init__(self, numberOfStages=numberOfStages, inf=inf)

        HydroelectricProduction.__init__(self, efficiency=0.88, gravity=10, minReservatoryVolume=12800,
            maxReservatoryVolume=21200, minTurbineFlow=1400, maxTurbineFlow=7955, maxProductionCapacity=3230,
            uprightPolinomy=lambda x: (303.04+(0.0015519*x) - (0.17377e-7)*(x**2)),
            downstreamPolinomy=lambda x: (279.84 +(0.22130e-3)*x))

        self.__maxFlow = maxFlow
        self.__year = year
        self.__stateSampling = stateSampling
        self.__decisionSampling = decisionSampling
    
    def monthlyAvgFlow(self, month:int) -> float:
        """Returns the average flow rate[m^3/s] at each month"""
        monthlyAvgFlowMap = {
            0: 9107,
            1: 7688,
            2: 9358,
            3: 6794,
            4: 4303,
            5: 3533,
            6: 2867,
            7: 2557,
            8: 2171,
            9: 2247,
            10: 3517,
            11: 4180 
            }
        if (month in monthlyAvgFlowMap):
            return monthlyAvgFlowMap.get(month)
        else:
            raise KeyError(f"dont exist month {month}")
    
    def monthlyPowerDemand(self, month:int) -> float:
        """Returns the expected demand of each month"""
        
        monthlyPowerDemandMap = {
            0: 2800,
            1: 2500,
            2: 2300,
            3: 2500,
            4: 2600,
            5: 2800,
            6: 2800,
            7: 2600,
            8: 2600,
            9: 2800,
            10: 3000,
            11: 3100 
            }
        
        if (month in monthlyPowerDemandMap):
            return monthlyPowerDemandMap.get(month)
        else:
            raise KeyError(f"dont exist month {month}")
        
        

    def thermalProductionCost(self, energy) -> float:
        return 64.8 * energy*energy*1e-6

    # ---------------------------------------------------------------------------------------------------------------    

    def state(self, k:int) -> np.array:
        return sampling(self.minReservatoryVolume(), self.maxReservatoryVolume(), self.__stateSampling)

    def decision(self, k:int) -> np.array:
        return sampling(self.minTurbineFlow(), self.__maxFlow, self.__decisionSampling)

    def elementaryCost(self, k:int, state:np.array, decision:np.array) -> float:
        demand = self.monthlyPowerDemand(k)
        generatedEnergy = self.generatedEnergy(state, decision)

        if generatedEnergy >= demand and generatedEnergy < self.maxProductionCapacity():
            cost = 0
        elif generatedEnergy>self.maxProductionCapacity():
            cost = self.thermalProductionCost(demand - self.maxProductionCapacity())
        else:
            cost = self.thermalProductionCost(demand - generatedEnergy)            
        
        return cost

    def finalStateCost(self, state:np.array) -> float:
        if state<15000:
            cost = self.inf
        else:
            cost = 0

        return cost

    def transitionFunction(self, k:int, state:np.array, decision:np.array) -> np.array:
        return state + (self.monthlyAvgFlow(k) - decision)*secondsOfMonth(self.__year, k)*1e-6
    
    def solveInfeasibility(self, k:int, state:np.array, FMap:dict):
        if state<self.minReservatoryVolume() or state>self.maxReservatoryVolume():
            return (limit(state, self.minReservatoryVolume(), self.maxReservatoryVolume()), self.inf)
        
        else:
            sampledState = nearestSample(state, self.minReservatoryVolume(), self.maxReservatoryVolume(), self.__stateSampling)
            return (sampledState, FMap[(k, sampledState)])

    def initialState(self) -> np.array:
        return 15000
    
    def solve(self):
        return solve(self.state, self.numberOfStages, self.finalStateCost, self.decision, self.solveInfeasibility,
        self.transitionFunction, self.elementaryCost, inf=self.inf)
    
    def optimalTrajectoryHidro(self, policy, initialState):
        nearestVolume = lambda vol: nearestSample(vol, self.minReservatoryVolume(), self.maxReservatoryVolume(), self.__stateSampling)
        transitionWithNearest = lambda k, state, decision: nearestVolume(self.transitionFunction(k, state, decision))

        return optimalTrajectory(nearestVolume(initialState), self.numberOfStages, policy, transitionWithNearest)
    
    def costOfSolution(self, initialState, FMap):
        nearestVolume = lambda vol: nearestSample(vol, self.minReservatoryVolume(), self.maxReservatoryVolume(), self.__stateSampling)

        return (nearestVolume(initialState), FMap[0, nearestVolume(initialState)])


def run(pairOfSamplings):

    _stateSampling, _decisionSampling = pairOfSamplings
    start = time()

    hidro = HydroelectricProductionProblem(numberOfStages=12, year=2021, maxFlow=10000, 
                stateSampling=_stateSampling, decisionSampling=_decisionSampling, inf=10000)

    fmap, policy = hidro.solve()
    u_optimal, policy_optimal = hidro.optimalTrajectoryHidro(policy, hidro.initialState())

    nearestInitial, cost = hidro.costOfSolution(hidro.initialState(), fmap)
    print(f"initialVolume: {nearestInitial}\nCost: {cost} M de reais\n")

    # print(f"\npolicy:\n{policy}")
    print(f"u_optimal: {u_optimal}")
    print(f"policy_optimal: {policy_optimal}")

    print(f"duration: {time() - start} \nsampling: state:{_stateSampling} x decision: {_decisionSampling}\n\n")
    print("-------------------------------------------")

if __name__ == '__main__':
    
    run((100, 100))

Stage 11 elapsed 0.20914053916931152 sec
Stage 10 elapsed 0.17383623123168945 sec
Stage 9 elapsed 0.18140029907226562 sec
Stage 8 elapsed 0.2005627155303955 sec
Stage 7 elapsed 0.18538141250610352 sec
Stage 6 elapsed 0.18352174758911133 sec
Stage 5 elapsed 0.1809709072113037 sec
Stage 4 elapsed 0.17420721054077148 sec
Stage 3 elapsed 0.18239545822143555 sec
Stage 2 elapsed 0.17885732650756836 sec
Stage 1 elapsed 0.19152092933654785 sec
Stage 0 elapsed 0.1808454990386963 sec
initialVolume: 15000
Cost: 1009.1829478332393 M de reais

u_optimal: [15000, 18500, 20900, 21100, 21100, 21100, 21200, 21100, 21000, 21200, 19500, 18000, 15000]
policy_optimal: [7800, 6700, 9300, 6800, 4300, 3500, 2900, 2600, 2100, 2900, 4100, 5300]
duration: 2.2378146648406982 
sampling: state:100 x decision: 100


-------------------------------------------


---

###### Usina de Ilha Solteira com afluências aleatórias

Para a solução deste problema foi criada a classe [HydroelectricStochasticProductionProblem](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/hydroelectricStochasticProductionProblem.py#L16)

In [7]:
from dypro.dypro.discreteDynamicProgramming import DiscreteDynamicProgramming
from dypro.dypro.hydroelectricProduction import HydroelectricProduction
from dypro.dypro.discreteDynamicProblem import DiscreteDynamicProblem
from dypro.dypro.stochasticProblem import StochasticProblem

from dypro.dypro.utils import limit, sampling, secondsOfMonth, nearestSample
from dypro.dypro.solve import solveStochastic
from dypro.dypro.randomVariable import RandomVariable

import numpy as np

from time import time

import concurrent.futures

class HydroelectricStochasticProductionProblem(HydroelectricProduction,  StochasticProblem, DiscreteDynamicProgramming):
    def __init__(self, numberOfStages:int, year:int, maxFlow, stateSampling=100, decisionSampling=100, inf=np.inf):

        HydroelectricProduction.__init__(self, efficiency=0.88, gravity=10, minReservatoryVolume=12800,
            maxReservatoryVolume=21200, minTurbineFlow=1400, maxTurbineFlow=7955, maxProductionCapacity=3230,
            uprightPolinomy=lambda x: (303.04+(0.0015519*x) - (0.17377e-7)*(x**2)),
            downstreamPolinomy=lambda x: (279.84 +(0.22130e-3)*x))

        DiscreteDynamicProgramming.__init__(self, numberOfStages=numberOfStages, inf=inf)

        self.__maxFlow = maxFlow
        self.__year = year
        self.__stateSampling = stateSampling
        self.__decisionSampling = decisionSampling

        flowDistribution = [
            [(6375, 0.1), (7741, 0.2), (9107, 0.4), (10473, 0.2), (11839, 0.1)], 
            [(5382, 0.1), (6535, 0.2), (7688, 0.4), (8841, 0.2), (9995, 0.1)],
            [(6550, 0.1), (7954, 0.2), (9358, 0.4), (10762, 0.2), (12165, 0.1)],
            [(4756, 0.1), (5775, 0.2), (6794, 0.4), (7814, 0.2), (8832, 0.1)],
            [(3012, 0.1), (3658, 0.2), (4303, 0.4), (4948, 0.2), (5594, 0.1)],
            [(2473, 0.1), (3003, 0.2), (3533, 0.4), (4063, 0.2), (4593, 0.1)],
            [(2007, 0.1), (2437, 0.2), (2867, 0.4), (3297, 0.2), (3727, 0.1)],
            [(1790, 0.1), (2173, 0.2), (2557, 0.4), (2941, 0.2), (3324, 0.1)],
            [(1520, 0.1), (1845, 0.2), (2171, 0.4), (2497, 0.2), (2822, 0.1)],
            [(1573, 0.1), (1910, 0.2), (2247, 0.4), (2584, 0.2), (2921, 0.1)],
            [(2462, 0.1), (2989, 0.2), (3517, 0.4), (4045, 0.2), (4572, 0.1)],
            [(2926, 0.1), (3553, 0.2), (4180, 0.4), (4807, 0.2), (5434, 0.1)]
            ]

        self.__realizableRandomValues = {k:RandomVariable([i[0] for i in el], [i[1] for i in el]) for k, el in enumerate(flowDistribution)}
    
    def realizableRandomValues(self, k:int) -> RandomVariable:
        return self.__realizableRandomValues[k]
    
    def monthlyPowerDemand(self, month:int) -> float:
        """Returns the expected demand of each month"""
        
        monthlyPowerDemandMap = {
            0: 2800,
            1: 2500,
            2: 2300,
            3: 2500,
            4: 2600,
            5: 2800,
            6: 2800,
            7: 2600,
            8: 2600,
            9: 2800,
            10: 3000,
            11: 3100 
            }
        
        if (month in monthlyPowerDemandMap):
            return monthlyPowerDemandMap.get(month)
        else:
            raise KeyError(f"dont exist month {month}")
    
    def thermalProductionCost(self, energy) -> float:
        return 64.8 * energy*energy*1e-6



    def state(self, k:int) -> np.array:
        return sampling(self.minReservatoryVolume(), self.maxReservatoryVolume(), self.__stateSampling)

    def decision(self, k:int) -> np.array:
        return sampling(self.minTurbineFlow(), self.__maxFlow, self.__decisionSampling)

    def elementaryCost(self, k:int, state:np.array, decision:np.array) -> float:
        demand = self.monthlyPowerDemand(k)
        generatedEnergy = self.generatedEnergy(state, decision)

        if generatedEnergy >= demand and generatedEnergy < self.maxProductionCapacity():
            cost = 0
        elif generatedEnergy>self.maxProductionCapacity():
            cost = self.thermalProductionCost(demand - self.maxProductionCapacity())
        else:
            cost = self.thermalProductionCost(demand - generatedEnergy)            
        
        return cost

    def finalStateCost(self, state:np.array) -> float:
        cost = 0.0247*(state-12800)*1e6

        return cost

    def transitionFunction(self, k:int, state:np.array, decision:np.array, randomValue:np.array) -> np.array:
        return state + (randomValue - decision)*secondsOfMonth(self.__year, k)*1e-6
    
    def solveInfeasibility(self, k:int, state:np.array, FMap:dict):
        if state<self.minReservatoryVolume() or state>self.maxReservatoryVolume():
            return (limit(state, self.minReservatoryVolume(), self.maxReservatoryVolume()), self.inf)
        
        else:
            sampledState = nearestSample(state, self.minReservatoryVolume(), self.maxReservatoryVolume(), self.__stateSampling)
            return (sampledState, FMap[(k, sampledState)])

    def initialState(self) -> np.array:
        return 15000
    
    def solve(self):
        return solveStochastic(self.state, self.numberOfStages, self.finalStateCost, self.decision, self.realizableRandomValues, self.solveInfeasibility,
        self.transitionFunction, self.elementaryCost, inf=self.inf)
    
    def costOfSolution(self, initialState, FMap):
        nearestVolume = lambda vol: nearestSample(vol, self.minReservatoryVolume(), self.maxReservatoryVolume(), self.__stateSampling)

        return (nearestVolume(initialState), FMap[0, nearestVolume(initialState)])
        
    

if __name__ == "__main__":

    start = time()
    h = HydroelectricStochasticProductionProblem(numberOfStages=12, year=2021, maxFlow=15000, stateSampling=50, decisionSampling=50, inf=100000)
    
    fmap, policy = h.solve()

    nearestInitial, cost = h.costOfSolution(h.initialState(), fmap)

    print(f"initialVolume: {nearestInitial}\nCost: {cost} M de reais\n")


    print(f"duration: {time() - start}")
    print("-------------------------------------------")

Stage 11 elapsed 2.7847084999084473 sec
Stage 10 elapsed 2.892540693283081 sec
Stage 9 elapsed 2.754714012145996 sec
Stage 8 elapsed 2.5895049571990967 sec
Stage 7 elapsed 2.6501450538635254 sec
Stage 6 elapsed 2.734217643737793 sec
Stage 5 elapsed 2.775862216949463 sec
Stage 4 elapsed 2.7933850288391113 sec
Stage 3 elapsed 2.806295871734619 sec
Stage 2 elapsed 2.8009963035583496 sec
Stage 1 elapsed 2.7377827167510986 sec
Stage 0 elapsed 2.8656628131866455 sec
initialVolume: 15000
Cost: 96557.7913876388 M de reais

duration: 33.21770930290222
-------------------------------------------


Todas as possíveis medidas encontram-se abaixo

In [10]:
print_map = dict()

for key in policy:
    if policy[key] != None:
        if key[0] not in print_map:
            print_map[key[0]] = {key:policy[key]}
        else:
            print_map[key[0]][key] = policy[key]

for stage in print_map:
    print(f"{stage}")
    print(print_map[stage])
    print("\n\n")

11
{(11, 12850): 5450, (11, 13000): 5500, (11, 13250): 5600, (11, 13400): 5650, (11, 13650): 5750, (11, 13800): 5800, (11, 14050): 5900, (11, 14200): 5950, (11, 14450): 6050, (11, 14600): 6100, (11, 15000): 6250, (11, 15400): 6400, (11, 15800): 6550, (11, 16200): 6700, (11, 16600): 6850, (11, 16750): 6900, (11, 17000): 7000, (11, 17150): 7050, (11, 17400): 7150, (11, 17550): 7200, (11, 17800): 7300, (11, 17950): 7350, (11, 18200): 7450, (11, 18350): 7500, (11, 18750): 7650, (11, 19150): 7800, (11, 19550): 7950, (11, 19950): 8100, (11, 20350): 8250, (11, 20500): 8300, (11, 20750): 8400, (11, 20900): 8450, (11, 21150): 8550}



10
{(10, 12800): 3500, (10, 12850): 1850, (10, 12900): 2400, (10, 12950): 3100, (10, 13000): 3050, (10, 13050): 3600, (10, 13100): 3000, (10, 13150): 2650, (10, 13200): 1750, (10, 13250): 3150, (10, 13300): 3700, (10, 13350): 3100, (10, 13400): 3200, (10, 13450): 3750, (10, 13500): 2100, (10, 13550): 2650, (10, 13600): 3350, (10, 13650): 3300, (10, 13700): 3850, (

---

###### Usina de Água Vermelha e Ilha Solteira 

Para a solução deste problema não foram reaproveitadas as classes anteriores, mas foram criadas classes que implementam as usinas: [HydroelectricAguaVermelha](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/hydroelectricProductionProblem2.py#L11) e [HydroelectricIlhaSolteira](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/hydroelectricProductionProblem2.py#L99); e a que implementa o problema: [HydroelectricProductionProblem2](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/hydroelectricProductionProblem2.py#L186).

In [11]:
from dypro.dypro.utils import limit, sampling, secondsOfMonth, nearestSample
from dypro.dypro.solve import solve, optimalTrajectory
import numpy as np
import logging
from time import time

import concurrent.futures

logging.basicConfig(level=logging.DEBUG)

class HydroelectricAguaVermelha:
    def __init__(self, stateSampling=100, decisionSampling=100, inf=np.inf):
        self.__efficiency = 0.88
        self.__gravity = 10
        self.__minReservatoryVolume = 4400 #10^6 m^3
        self.__maxReservatoryVolume = 11000 #10^6 m^3
        self.__minTurbineFlow = 475 #m^3/s
        self.__maxTurbineFlow = 2710 #m^3/s
        self.__maxFlow = 5000 #m^3/s
        self.__year = 2021
        self.__numberOfStages = 11
        self.__stateSampling = stateSampling
        self.__decisionSampling = decisionSampling
        self.__inf = inf
    
    def monthlyAvgFlow(self, month:int) -> float:
        """Returns the average flow rate[m^3/s] at each month"""
        monthlyAvgFlowMap = {
            0: 3899,
            1: 3202,
            2: 2953,
            3: 2600,
            4: 1671,
            5: 1271,
            6: 1045,
            7: 972,
            8: 792,
            9: 790,
            10: 1139,
            11: 1589
            }
        if (month in monthlyAvgFlowMap):
            return monthlyAvgFlowMap.get(month)
        else:
            raise KeyError(f"dont exist month {month}")      

    def maximumProductionCapacity(self) -> float:
        return 3230.0
    
    def rho(self) -> float:
        return self.__efficiency * self.__gravity * 1e-3
    
    def uprightHeight(self, reservatoryVolume:float) -> float:
        limitedVolume = limit(reservatoryVolume, self.__minReservatoryVolume, self.__maxReservatoryVolume)
        
        return 355.53 + (0.0036268 * limitedVolume) - (0.10090e-7)*limitedVolume**2

    def downstreamHeight(self, turbinedFlow:float) -> float:
        return 319.91 + (0.15882e-2) * turbinedFlow
    
    def generatedEnergy(self, reservatoryVolume:float, turbinedFlow:float):
        flowThatGenerateEnergy = limit(turbinedFlow, self.__minTurbineFlow, self.__maxTurbineFlow)

        _generatedEnergy = min(self.maximumProductionCapacity(), self.rho() * (self.uprightHeight(reservatoryVolume) - self.downstreamHeight(turbinedFlow)) * flowThatGenerateEnergy)

        return _generatedEnergy
    
    def state(self, k:int) -> np.array:
        return sampling(self.__minReservatoryVolume, self.__maxReservatoryVolume, self.__stateSampling)
    
    def decision(self, k:int) -> np.array:
        return sampling(self.__minTurbineFlow, self.__maxFlow, self.__decisionSampling)
    
    def solveInfeasibility(self, k:int, state:np.array):
        """[summary]

        Args:
            k (int): [description]
            state (np.array): [description]
            FMap (dict): [description]

        Returns:
            [type]: (next state, isInfeasible)
        """
        isInfeasible = state<self.__minReservatoryVolume or state>self.__maxReservatoryVolume
      
        if isInfeasible:
            feasibleState = limit(state, self.__minReservatoryVolume, self.__maxReservatoryVolume)
        
        else:
            feasibleState = nearestSample(state, self.__minReservatoryVolume, self.__maxReservatoryVolume, self.__stateSampling)
            
        return (feasibleState, isInfeasible)
    
    def nearestVolume(self, volume):
        return nearestSample(volume, self.__minReservatoryVolume, self.__maxReservatoryVolume, self.__stateSampling)


class HydroelectricIlhaSolteira:
    def __init__(self, stateSampling=100, decisionSampling=100, inf=np.inf):
        self.__efficiency = 0.89
        self.__gravity = 10
        self.__minReservatoryVolume = 12800 #10^6 m^3
        self.__maxReservatoryVolume = 21200 #10^6 m^3
        self.__minTurbineFlow = 1400 #m^3/s
        self.__maxTurbineFlow = 7955 #m^3/s
        self.__maxFlow = 10000 #m^3/s
        self.__year = 2021
        self.__numberOfStages = 11
        self.__stateSampling = stateSampling
        self.__decisionSampling = decisionSampling
        self.__inf = inf
    
    def monthlyAvgFlow(self, month:int) -> float:
        """Returns the average flow rate[m^3/s] at each month"""
        monthlyAvgFlowMap = {
            0: 5208,
            1: 4486,
            2: 6405,
            3: 4194,
            4: 2632,
            5: 2262,
            6: 1822,
            7: 1585,
            8: 1379,
            9: 1457,
            10: 2378,
            11: 2591
            }
        if (month in monthlyAvgFlowMap):
            return monthlyAvgFlowMap.get(month)
        else:
            raise KeyError(f"dont exist month {month}")      

    def maximumProductionCapacity(self) -> float:
        return 1380.0
    
    def rho(self) -> float:
        return self.__efficiency * self.__gravity * 1e-3
    
    def uprightHeight(self, reservatoryVolume:float) -> float:
        limitedVolume = limit(reservatoryVolume, self.__minReservatoryVolume, self.__maxReservatoryVolume)
        
        return 303.04 + (0.0015519 * limitedVolume) - (0.17377e-7)*limitedVolume**2

    def downstreamHeight(self, turbinedFlow:float) -> float:
        return 279.84 + (0.22130e-3) * turbinedFlow
    
    def generatedEnergy(self, reservatoryVolume:float, turbinedFlow:float):
        flowThatGenerateEnergy = limit(turbinedFlow, self.__minTurbineFlow, self.__maxTurbineFlow)
        
        _generatedEnergy = min(self.maximumProductionCapacity(), self.rho() * (self.uprightHeight(reservatoryVolume) - self.downstreamHeight(turbinedFlow)) * flowThatGenerateEnergy)

        return _generatedEnergy
    
    def state(self, k:int) -> np.array:
        return sampling(self.__minReservatoryVolume, self.__maxReservatoryVolume, self.__stateSampling)
    
    def decision(self, k:int) -> np.array:
        return sampling(self.__minTurbineFlow, self.__maxFlow, self.__decisionSampling)
    
    def solveInfeasibility(self, k:int, state:np.array):
        """[summary]

        Args:
            k (int): [description]
            state (np.array): [description]
            FMap (dict): [description]

        Returns:
            [type]: (next state, wasInfeasible)
        """
        wasInfeasible = state<self.__minReservatoryVolume or state>self.__maxReservatoryVolume
      
        if wasInfeasible:
            feasibleState = limit(state, self.__minReservatoryVolume, self.__maxReservatoryVolume)
        
        else:
            feasibleState = nearestSample(state, self.__minReservatoryVolume, self.__maxReservatoryVolume, self.__stateSampling)
            
        return (feasibleState, wasInfeasible)
    
    def nearestVolume(self, volume):
        return nearestSample(volume, self.__minReservatoryVolume, self.__maxReservatoryVolume, self.__stateSampling)

class HydroelectricProductionProblem2:
    def __init__(self, stateSampling=100, decisionSampling=100, inf=np.inf):
        self.__aguaVermelha = HydroelectricAguaVermelha(stateSampling, decisionSampling, inf)
        self.__ilhaSolteira = HydroelectricIlhaSolteira(stateSampling, decisionSampling, inf)
        self.stateSampling = stateSampling
        self.decisionSampling = decisionSampling
        self.__numberOfStages = 12
        self.__inf = inf
        self.__year = 2021
    
    def inf(self):
        return self.__inf
    
    def monthlyPowerDemand(self, month:int) -> float:
        """Returns the expected demand of each month"""
        
        monthlyPowerDemandMap = {
            0: 3600,
            1: 3300,
            2: 3000,
            3: 3200,
            4: 3400,
            5: 3600,
            6: 3700,
            7: 3300,
            8: 3400,
            9: 3600,
            10: 3900,
            11: 4000 
            }
        
        if (month in monthlyPowerDemandMap):
            return monthlyPowerDemandMap.get(month)
        else:
            raise KeyError(f"dont exist month {month}")
    
    def thermalProductionCost(self, energy) -> float:
        return 64.8 * energy*energy*1e-6
    
    def generatedEnergy(self, state, decision):

        gen_agua_vermelha = self.__aguaVermelha.generatedEnergy(state[0], decision[0])
        gen_ilha_solteira = self.__ilhaSolteira.generatedEnergy(state[1], decision[1])
        
        return gen_agua_vermelha + gen_ilha_solteira

    def maximumProductionCapacity(self):
        return self.__aguaVermelha.maximumProductionCapacity() + self.__ilhaSolteira.maximumProductionCapacity()
    
    # ---------------------------------------------------------------------------------------------------------------    

    def state(self, k:int) -> np.array:
        samples_agua_vermelha = list(self.__aguaVermelha.state(k))
        samples_ilha_solteira = list(self.__ilhaSolteira.state(k))

        return [(a, b) for a in samples_agua_vermelha for b in samples_ilha_solteira]

    def decision(self, k:int) -> np.array:
        samples_agua_vermelha = list(self.__aguaVermelha.decision(k))
        samples_ilha_solteira = list(self.__ilhaSolteira.decision(k))

        return [(a, b) for a in samples_agua_vermelha for b in samples_ilha_solteira]

    def elementaryCost(self, k:int, state:np.array, decision:np.array) -> float:
        demand = self.monthlyPowerDemand(k)
        generatedEnergy = self.generatedEnergy(state, decision)
        
        cost = self.thermalProductionCost(demand - self.maximumProductionCapacity())
                
        return cost

    def finalStateCost(self, state:np.array) -> float:
        agua_vermelha_vol = state[0]
        ilha_solteira_vol = state[1]

        if agua_vermelha_vol<8000 or ilha_solteira_vol<15000:
            cost = self.__inf
        else:
            cost = 0

        return cost

    def transitionFunction(self, k:int, state:np.array, decision:np.array) -> np.array:
        aguaVermelhaState = state[0] + (self.__aguaVermelha.monthlyAvgFlow(k) - decision[0])*secondsOfMonth(self.__year, k)*1e-6
        ilhaSolteiraState = state[1] + (self.__ilhaSolteira.monthlyAvgFlow(k) - decision[1])*secondsOfMonth(self.__year, k)*1e-6


        return (aguaVermelhaState, ilhaSolteiraState)
    
    def solveInfeasibility(self, k:int, state:np.array, FMap:dict):
        aguaVermelhaState, aguaVermelhaWasInfeasible = self.__aguaVermelha.solveInfeasibility(k, state[0])
        ilhaSolteiraState, ilhaSolteiraWasInfeasible = self.__ilhaSolteira.solveInfeasibility(k, state[1])

        if aguaVermelhaWasInfeasible or ilhaSolteiraWasInfeasible:
            return ((aguaVermelhaState, ilhaSolteiraState), self.__inf)
        
        else:
            return ((aguaVermelhaState, ilhaSolteiraState), FMap[(k, (aguaVermelhaState, ilhaSolteiraState))])

    def initialState(self) -> np.array:
        return (8000, 15000)
    
    def solveHidro(self):
        return solve(self.state, self.__numberOfStages, self.finalStateCost, self.decision, self.solveInfeasibility,
        self.transitionFunction, self.elementaryCost, inf=self.__inf)
    
    def optimalTrajectoryHidro(self, policy, initialState):
        transitionWithNearest = lambda k, state, decision: self.nearestVolume(self.transitionFunction(k, state, decision))

        return optimalTrajectory(self.nearestVolume(initialState), self.__numberOfStages, policy, transitionWithNearest)
    
    def nearestVolume(self, state):
        return (self.__aguaVermelha.nearestVolume(state[0]), self.__ilhaSolteira.nearestVolume(state[1]))
    
    def costOfSolution(self, initialState, FMap):
        return (self.nearestVolume(initialState), FMap[0, self.nearestVolume(initialState)])

In [12]:
if __name__ == "__main__":
    h = HydroelectricProductionProblem2(200, 200, 1000000)

    Fmap, policy = h.solveHidro()

    # Fmap = {key:Fmap[key] for key in Fmap if Fmap[key]<h.inf()}

    nearestInitial, cost = h.costOfSolution(h.initialState(), Fmap)
    print(f"initialVolume: {nearestInitial}\nCost: {cost} M de reais\n")


    u_optimal, policy_optimal = h.optimalTrajectoryHidro(policy, nearestInitial)

    print(f"u_optimal: {u_optimal}")
    print(f"policy_optimal: {policy_optimal}")

    # print(Fmap)
    # print("\n\n\n\n")
    # print(policy)

Stage 11 elapsed 56.08609318733215 sec
Stage 10 elapsed 58.17532515525818 sec
Stage 9 elapsed 58.87287712097168 sec
Stage 8 elapsed 60.14893436431885 sec
Stage 7 elapsed 63.54301619529724 sec
Stage 6 elapsed 67.81258463859558 sec
Stage 5 elapsed 66.22877764701843 sec
Stage 4 elapsed 69.37238550186157 sec
Stage 3 elapsed 68.83258938789368 sec
Stage 2 elapsed 70.9524393081665 sec
Stage 1 elapsed 74.19974613189697 sec
Stage 0 elapsed 72.61327934265137 sec
initialVolume: (8000, 15000)
Cost: 1017.69696 M de reais

u_optimal: [(8000, 15000), (10800, 21000), (10600, 20800), (10800, 20800), (10600, 20800), (10600, 20800), (10600, 21000), (10600, 21000), (10800, 21000), (10600, 21000), (11000, 21200), (10600, 21200), (11000, 21200)]
policy_optimal: [(2875, 3000), (3275, 4600), (2875, 6400), (2675, 4200), (1675, 2600), (1275, 2200), (1075, 1800), (875, 1600), (875, 1400), (675, 1400), (1275, 2400), (1475, 2600)]
